In [1]:
# import base64
# import imageio
from hangman_guesser.env.TFenv import HangmanEnvironment

import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf

from tf_agents.agents.dqn.dqn_agent import DqnAgent, DdqnAgent
from tf_agents.networks.q_network import QNetwork

from tf_agents.environments import suite_gym


from tf_agents.environments import tf_py_environment

from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.replay_buffers.tf_uniform_replay_buffer import TFUniformReplayBuffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common
from multiprocessing import Pool



In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# print(device_lib.list_local_devices())


Num GPUs Available:  0


In [2]:
# Globals
NUMBER_EPOSODES = 1000
COLLECTION_STEPS = 1
BATCH_SIZE = 64
EVAL_EPISODES = 10
EVAL_INTERVAL = 50

dictionary_path = "../../../properties/words_250000_train.txt"
train_env = HangmanEnvironment(dictionary_path)
evaluation_env = HangmanEnvironment(dictionary_path)

print('Observation Spec:')
print(train_env.time_step_spec().observation)

print('Reward Spec:')
print(train_env.time_step_spec().reward)

print('Action Spec:')
print(train_env.action_spec())

train_env = tf_py_environment.TFPyEnvironment(train_env)
evaluation_env = tf_py_environment.TFPyEnvironment(evaluation_env)




Observation Spec:
BoundedArraySpec(shape=(1, 30), dtype=dtype('int32'), name='game', minimum=-1, maximum=26)
Reward Spec:
ArraySpec(shape=(), dtype=dtype('float32'), name='reward')
Action Spec:
BoundedArraySpec(shape=(), dtype=dtype('int32'), name='letter', minimum=0, maximum=25)


In [13]:
# train_env.action_spec()
train_env.time_step_spec()
log_interval = 200

TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(1, 30), dtype=tf.int32, name='game', minimum=array(-1, dtype=int32), maximum=array(26, dtype=int32)))

In [4]:

hidden_layers = (100,)

# dqn_network = QNetwork(
#     train_env.observation_spec(),
#     train_env.action_spec(),
#     fc_layer_params=hidden_layers)

ddqn_network = QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=hidden_layers)

counter = tf.Variable(0)

ddqn_agent = DdqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network = ddqn_network,
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-3),
    td_errors_loss_fn = common.element_wise_squared_loss,
    train_step_counter = counter)

ddqn_agent.initialize()

In [ ]:

def get_average_reward(environment, policy, episodes=10):

    total_reward = 0.0

    for _ in range(episodes):
        time_step = environment.reset()
        episode_reward = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_reward += time_step.reward
            print(time_step)
    
        total_reward += episode_reward
        avg_reward = total_reward / episodes
    
    return avg_reward.numpy()[0]

class ExperienceReplay(object):
    def __init__(self, agent, enviroment):
        self._replay_buffer = TFUniformReplayBuffer(
            data_spec=agent.collect_data_spec,
            batch_size=enviroment.batch_size,
            max_length=50000)
        
        self._random_policy = RandomTFPolicy(train_env.time_step_spec(),
                                                enviroment.action_spec())
        
        self._fill_buffer(train_env, self._random_policy, steps=100)
        
        self.dataset = self._replay_buffer.as_dataset(
            num_parallel_calls=3, 
            sample_batch_size=BATCH_SIZE, 
            num_steps=2).prefetch(3)

        self.iterator = iter(self.dataset)
    
    def _fill_buffer(self, enviroment, policy, steps):
        for _ in range(steps):
            self.timestamp_data(enviroment, policy)
            
    def timestamp_data(self, environment, policy):
        time_step = environment.current_time_step()
        action_step = policy.action(time_step)
        next_time_step = environment.step(action_step.action)
        timestamp_trajectory = trajectory.from_transition(time_step, action_step, next_time_step)

        self._replay_buffer.add_batch(timestamp_trajectory)




Num GPUs Available:  0


In [ ]:
# def train(agent):
print("hello")
experience_replay = ExperienceReplay(ddqn_agent, train_env)
print("hello")

ddqn_agent.train_step_counter.assign(0)
print("hello")

avg_reward = get_average_reward(evaluation_env, ddqn_agent.policy, EVAL_EPISODES)
rewards = [avg_reward]
print("hello")

for _ in range(NUMBER_EPOSODES):
    print("hello")
    for _ in range(COLLECTION_STEPS):
        experience_replay.timestamp_data(train_env, ddqn_agent.collect_policy)

    experience, info = next(experience_replay.iterator)
    train_loss = ddqn_agent.train(experience).loss
    print("hello")

    print(ddqn_agent.train_step_counter.numpy())
    if ddqn_agent.train_step_counter.numpy() % EVAL_INTERVAL == 0:
        avg_reward = get_average_reward(evaluation_env, ddqn_agent.policy, EVAL_EPISODES)
        print('Episode {0} - Average reward = {1}, Loss = {2}.'.format(
                ddqn_agent.train_step_counter.numpy(), avg_reward, train_loss))
        rewards.append(avg_reward)
            
    # return rewards, agent, experience_replay

# print("**********************************")
# print("Training DQN")
# print("**********************************")
# dqn_reward = train(dqn_agent)

print("**********************************")
print("Training DDQN")
print("**********************************")
# ddqn_reward, agent = train(ddqn_agent)


In [11]:
# # train_env.render()
import logging
logging.basicConfig(format='%(levelname)s:%(message)s',
                    level=logging.DEBUG)
ddqn_agent.train_step_counter.numpy()
# time_step = train_env.current_time_step()
# print(time_step)
# print("".join([chr(int(x)+97) for x in time_step[3][0][0]]).replace("{",".").replace('`',""))

# action_step = ddqn_agent.policy.action(time_step)
# action_step

0

In [ ]:
episodes = range(0, NUMBER_EPOSODES + 1, EVAL_INTERVAL)
plt.plot(episodes, ddqn_reward, label = 'DDQN Rewards')
plt.legend()
plt.ylabel('Average reward')
plt.xlabel('Epoisodes')
plt.ylim(top=50)


In [ ]:
# # save policy
# PolicySaver(eval_policy).save(model_dir + 'eval_policy')
# ...
# # load policy
# policy = tf.saved_model.load(model_path)
# action = policy.action(timestep)


In [ ]:
tempdir = os.getenv("TEST_TMPDIR", tempfile.gettempdir())
checkpoint_dir = os.path.join(tempdir, 'checkpoint')
train_checkpointer = common.Checkpointer(
    ckpt_dir=checkpoint_dir,
    max_to_keep=1,
    agent=agent,
    policy=agent.policy,
    replay_buffer=ExperienceReplay,
)
train_checkpointer.save()



In [ ]:
policy_dir = os.path.join(tempdir, 'policy')
tf_policy_saver = policy_saver.PolicySaver(agent.policy)
tf_policy_saver.save(policy_dir)
